<a href="https://colab.research.google.com/github/davis689/binder/blob/master/CHEM461/Numerical_solutions_to_Schrodinger_Equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Numerically Solving the Schrödinger Equation
(Some of the following (mostly, I think, the finite differences procedure) was taken from someone else's Jupyter notebook and I can't remember whose. Credit to that unknown creator.)

The number of problems for which we will be able to find an analytical solution to the Schrödinger equation is limited. We have seen how to solve the particle in a box and we will be able to analytically solve some others but even in those cases we often rely on solutions by long dead French mathematicians rather than doing the tedious work ourselves.

We should use the cases in which we can find a precise solution to develop our intuition about what a reasonable solution looks like and be able to make informed qualitative guesses at what a good wavefunction will be. But it would also be useful to know techniques in calculating the wavefunctions and energies numerically. Here our answers may not be precise but we with enough work we should be able to approach the true answer as closely as we want.

We have seen the Schrödinger Equation written as a differential equation and we have seen that it forms an eigenfunction/eigenvalue equation.
$$ -\frac{\hbar^2}{2m}\frac{d^2}{d x^2} \psi(x) + V(x)\psi(x) = E\psi(x)$$

Here we will see how to transform the equation into a matrix equation that preserves the eigenfunction/eigenvalue form (where the term $eigenfunction$ is replaced by $eigenvector$).


## Method
The method we will use here is called the ["Finite Difference Method"](https://en.wikipedia.org/wiki/Finite_difference_method) (see the linked Wikipedia article). In this method we will turn the function $\psi(x)$ into a vector, which is a list in Python, and the operator of the differential equation into a *matrix*. We then end up with a matrix eigenequation, which we can diagonalize to get our answer.

### Discretization
The process of discretization is simply turning our continuous space $x$, into a discrete number of steps, $N$, and our function $\psi(x)$ into an array of size $N$. We thus have $N$ values $x_i$, which have a stepsize $h = \Delta x = x_{i+1} - x_i$. Our choice of the *size* of our space, $N$, turns out to be important. Too large a number will slow down our computation and require too much computer memory, too small a number and the answers we compute will not be sufficiently accurate. A common practice is to start with a small number $N$ and then increase it until the accuracy is acceptable. The actual value you obtain in the end will depend on the problem you are studying.

### first order differential
We first need to develop how we will take the derivative of our function. Going back to our introduction to calculus, we remember that the derivative was defined as:
$$
\frac{d}{dx} f(x) = \lim_{\Delta x \rightarrow 0} \frac{f(x+\Delta x) - f(x)}{\Delta x} \approx \frac{f(x+h) - f(x)}{h} + \mathrm{O}(h)
$$
When we take a *finite difference*, we simply not take the limit all the way down to 0, but stop at $\Delta x = h$. Note that for this equation we evaluate the point just *after* $x$, which we call the *forward difference*. There are other possible ways to do a finite difference but we won't concern ourselves with them here.

Note that for this approximation to a derivative, we have a problem at the end of our space. In our case, $x+\Delta x$ is not defined at the last point. To account for this, we'll have to ignore the last point of our derivative.

#### Matrix representation
Matrix and vector methods are very straightforward to implement using a computer. We need to see how a matrix can be setup to take a derivative. I expect that many of you will not have taken linear algebra and we won't need to concern ourselves too much with each detail of the matrix but you should be able to see the main point. You can look at the[ matrix multiplication section ](#scrollTo=xsj8ZRIiPpNr&line=10&uniqifier=1)to get a little background that may be useful.


We'll want to find a matrix that can turn our vector representation of $\psi$ into its derivative.
We now want to turn our Schrödinger equation into a matrix equation for us to evaluate. If you just want to take the derivative of a function stored in an array, then this is not needed, you can run a loop and evaluate the equation for each $x_i$, but if you want to solve a differential equation, we need the matrix.

We introduce the vectors $f(x) = [f_0,f_1,f_2,...,f_{N-1}]$ and for the derivative $f'(x) = [f'_0,f'_1,f'_2...,f'_{N-1}]$. The forward difference derivative can then be written as:
$$ f'_i = (f_{i+1} - f_i)/h $$
And the matrix equation for the forward difference derivative is just:
$$
\begin{pmatrix}f'_0 \\ f'_1 \\\vdots \\ f'_{N-1}\end{pmatrix} = \frac{1}{h}\begin{pmatrix}f_1-f_0\\f_2-f_1\\\vdots\\f_{N}-f_{N-1}\end{pmatrix}= \frac{1}{h}
\begin{pmatrix} -1 & 1 & 0 & & \\ 0 & -1 & 1 & & \\ & & \ddots & \ddots \\
& & & -1 & \end{pmatrix}\begin{pmatrix}f_0 \\ f_1 \\\vdots \\ f_{N-1}\end{pmatrix}
$$

The matrix only has two non-zero elements on each row. This means that only the two corresponding elements in the function vector ($f_0$, $f_1$, $f_2$, $f_3$ ...) will be affected by the multiplication of each row. So, ignoring the division by $h$, the first row of the resulting vector will be $-1\cdot f_0 + 1\cdot f_1 + 0\cdot f_2 + 0\cdot... = f_1-f_0$

We note that the last entry in the matrix will not be correct because there is no element for $N$. We will need to remember to ignore this last point in our calculation of the derivative.

So the multiplication of the derivative matrix by our function returns the difference between each adjacent point in our function. After this the derivative is obtained by dividing each element by the difference between each point on the $x$-axis, here denoted by $h$. As long as this distance is constant, we can simply divide each element by $h$ to get the derivative.

We now check that this works. Using Python and Numpy we will take the derivative of $\sin(x)$ and plot the result.
First we tell the system we want to use the Numpy package to deal with numerical calculations and the matplotlib package to deal with making plots.
Then setup some parameters such as number of points, the $x$-axis range, etc.

### Example derivative calculation
#### Small matrices
We'll calculate the derivative using a matrix like the one above of $y=2x$ over some small range of $x$ as a way of testing the technique while using functions for which we can easily see whether the technique is working.

In [1]:
import numpy as np # import numerical computation functions
import matplotlib.pyplot as plt # import plotting functions

In [2]:
N=6 # chop the x domain in to N pieces. Keep it small for fast calculations, large for more accuracy
x=np.linspace(0,5,N) # set up the array of x values. Begin at the first number, end at the second value, and return N evenly spaced numbers.

Δx = x[1]-x[0] # name a variable for the spacing between the x values

We can use generate our matrix using the numpy [diag and ones](#scrollTo=5W2m_ana1yrl&line=2&uniqifier=1) functions

In [ ]:
D=(np.diag(-1*np.ones(N),0)+np.diag(np.ones(N-1),1))/Δx #define derivative matrix put -1 on the diagonal and 1 on the at all points just above the diagonal

y=2*x # this is the function we want to differentiate
yprime=D@y #matrix multiply D and y. D.dot(y) or np.matmul(D,y) also accomplishes the multiplication. D*y does not.
yprime=yprime[:-1] # :-1 means exclude the last point as there is no point f(x+Δx) to evaluate.

print("The derivative is ",yprime)

plt.plot(x,y,label='y(x)')
plt.plot(x[:-1],yprime,label='dy/dx)')  # we have to leave off the last element of x to make it the same size as the derivative.
                                        #Leave off [:-1] here and in yprime definition to see what happens if you keep all the points.
plt.legend()
plt.show()


You should see that the derivative is a constant, 2.

Try changing $y(x)$ to $x^2$ (which you express in python as ```x**2```). The derivative here should still be linear but with a non-zero slope.

Try $y(x)=x^3$. You should notice that the curves are made up of line segments. Increasing the value of $N$ will create smoother curves and a more accurate derivative. Try N=50.

Now try $y(x)=sin(x)$. Type ```np.sin(x)``` to get the $sin$ function since it comes from the numpy library.

### Second order Differential
We can now extend this method to the second order differential. Since we did a 'forward' first derivative (adding Δx), it will be convenient for us to do a ['backward'](#scrollTo=DkcuHT4C-bpw&line=2&uniqifier=1) second derivative here:
$$
\frac{d^2}{dx^2}f(x) = \lim_{\Delta x \rightarrow 0} \frac{f'(x)-f'(x-\Delta x)}{\Delta x} =  \lim_{\Delta x \rightarrow 0} \frac{f(x+\Delta x) - f(x) - (f(x) - f(x-\Delta x))}{\Delta x^2} \\
\frac{d^2}{dx^2}f(x) = \lim_{\Delta x \rightarrow 0} \frac{f(x+\Delta x) - 2f(x) + f(x-\Delta x))}{\Delta x^2} \approx \frac{f(x+\Delta x) - 2f(x) + f(x-\Delta x))}{\Delta x^2}
$$
So in the discrete space we can write this as:
$$ f''_i = (f_{i+1} - 2f_i + f_{i-1})/\Delta x^2 $$
And finally, as a matrix equation, the second derivative is then:
$$
\begin{pmatrix}f''_0 \\ f''_1 \\ f''_2 \\\vdots \\ f''_{N-1}\end{pmatrix} = \frac{1}{\Delta x^2}
\begin{pmatrix} -2 & 1 & 0 & 0 & \\ 1 & -2 & 1 & 0 & \\
0& 1 & -2 & 1 &  \\ &  & \ddots & \ddots & \ddots &\\
&  & & 1 & -2 \end{pmatrix}\begin{pmatrix}f_0 \\ f_1 \\ f_2 \\\vdots \\ f_{N-1}\end{pmatrix}
$$
Where now we note that at both ends of our array we will get an inaccurate answer unless we do some fixup. The fixup in this case is to use the same elements as the row below (at the start) or the row above (at the end), so we get $f''_0 = f''_1$ and $f''_{N-1} = f''_{N-2}$, which is not great but better than the alternative.

We can now try this matrix in Python and compute the second derivative of our $y(x)$ array.

In [4]:
N=200
x=np.linspace(0,2*np.pi,N)
Δx=x[1]-x[0]

y=np.sin(x)

D1=(np.diag(-1*np.ones(N),0)+np.diag(np.ones(N-1),1))/Δx

D2 = (np.diag(np.ones(N-1),-1) + np.diag( -2.*np.ones(N),0) + np.diag(np.ones(N-1),1))/(Δx**2)
                # The first np.diag() puts (N-1) ones on the partial diagonal below the diagonal (,-1)
                # The second puts N negative twos on the diagonal. The third puts (N-1) ones on the partial diagonal above the diagonal (,1).
yp=D1.dot(y) # take derivative so we can use it in our plot
ypp = D2.dot(y) # take second derivative of function, y.



At this point we have the first and second derivatives. Let's plot them.

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(x,y,label='y(x)',color='blue')
plt.plot(x[:-1],yp[:-1],label='y\'(x)',color='orange')     # Last value is invalid, don't plot
plt.plot(x[1:-1],ypp[1:-1],label='y\'\'(x)',color='green')  # First and last value is invalid.
plt.legend()
plt.show()

Do these derivatives of sine lead to the result you expect?

# Solving the Particle in a Box Schrödinger Equation

We can now setup the Schrodinger Equation as a matrix equation:
$$
\hat H = \frac{\hbar^2}{2m}\frac{d^2}{d x^2} + V \\
\hat H \psi(x) = E \psi(x)
$$
We now know the matrix for taking the second order derivative. The matrix for the potential is simply the values of the potential on the diagonal of the matrix: $\mathbf{V}_{i=j} = V_i$. The result of multiplying the function vector will be to simply multiply each element of the function by the value of the potential.

Writing out the matrix for $\mathbf{H}$ we get:
$$
\mathbf{H} = \frac{-\hbar^2}{2 m \Delta x^2} \begin{pmatrix} -2 & 1 & 0 & 0 & \\ 1 & -2 & 1 & 0 & \\
0& 1 & -2 & 1 &  \\ &  & \ddots & \ddots & \ddots &\\
&  & & 1 & -2 \end{pmatrix} +
\begin{pmatrix} V_0 & 0 & 0 & & \\ 0 & V_1 & 0 & & \\ 0 & 0 & V_2 & & \\ & & &\ddots & \\ &&&&V_{N-1}\end{pmatrix}
$$

It is worth looking at the matrix of the Hamiltonian and notice the symmetry: $\mathbf{H}^T = \mathbf{H}$, so the transpose of the matrix is identical to the matrix. Since the matrix is *real* everywhere, the complex conjugate is also the same: $\mathbf{H}^*=\mathbf{H}$. Combining these two statements, we can say that the Hamiltonian is Hermetian: $\mathbf{H}^\dagger = \mathbf{H}$.



### Infinite Square Well
The very simplest system to solve for is the infinite square well, for which $V=0$. We will readily recognize the results as alternating $\cos(x)$ and $\sin(x)$ functions, and the energy levels are:
$$
E_i = \frac{n^2\pi^2\hbar^2}{2ma^2}
$$
First, we need to discuss a subtlety. The Infinite Square Well from $-a/2$ to $a/2$ has $V=\infty$ *at* these points. We get into trouble trying to enter $\infty$ in our potential, so what we need to do is just limit the computational space from $-a/2+h$ to $a/2-h$, where $h$ is our step size. That way we force the wavefunction to zero at the end points.
We compute this in the next box. I create $x_{full}$ as the full x-axis from $-a/2$ to $a/2$, but take $N+2$ steps. I then leave out the first and last point when calculating the wavefunctions. At the end, before plotting, I add a zero to the beginning and end of the wavefunctions, so that we get the expected result for plotting.

Note I again import everything and setup all the definitions, so this block is stand-alone, and can be copy-pasted into another notebook. Here we set up constants and import the library needed for dealing with linear algebra.

In [6]:
hbar=1 # use atomic units
m=1 # electron mass

N = 1000
a = 1.0 # this doesn't matter unless we want to compare different size boxes.
x = np.linspace(-a/2.,a/2.,N)

Δx = x[1]-x[0] # This is the difference between points.


Now we set up the potential energy, the second derivative operator, and the Hamiltonian. Then extract the eigenvalues and eigenfunctions.

In [7]:
V = 0.*x # could be just zero but put x here to show that it is a function of x.
D2 = 1./(Δx**2)*(np.diag(np.ones(N-1),-1) -2* np.diag(np.ones(N),0) + np.diag(np.ones(N-1),1)) # derivative operator
H = -(hbar*hbar)/(2.0*m)*D2 + np.diag(V)  # Hamiltonian including derivative and potential energy.


E,psiT = np.linalg.eigh(H) # This computes the eigenvalues and eigenvectors
psi = np.transpose(psiT)   # We take the transpose of psiT to the wavefunction vectors can accessed as psi[n]

The energies are stored in $E$. The first $N$ levels are calculated. We will focus on only a few of them. Let's list the first several in atomic energy units.

In [ ]:
for i in range(50):
  print("E_{}={:8.3f}".format(i+1,E[i])) # the notation here is strange. Basically we will print everything between the ""
                                         #but stuff between the brackets will get subsitututed in from the between the parentheses in .format()
                                         #So here we'll print i+1 which is the quantum number 'n' in the particle in a box and the energies.
                                         #The :8.3f stuff indicates how many places to use for the whole number (some may be spaces) and how many for the
                                         #after the decimal point. The 'f' indicates a floating point (or decimal number). If we don't care about the
                                         #format of the number that is displayed or if the number is a whole number as it is, like 'i', just {} is fine.

####Scaling the energies
The energy of the lowest level in a particle in a box is $\frac{n^2 \hbar^2 \pi^2}{2 m a^2}$. For convenience we will rescale the energies to be relative to the energy of the first level. So $\epsilon=\frac{E_n}{E_1}=\frac{E_n}{\frac{\hbar^2 \pi^2}{2ma^2}}=\frac{E_n}{\frac{\pi^2}{2}}=\frac{2E_n}{\pi^2}$ and $ϵ=\frac{E_n}{\frac{\hbar^2 \pi^2}{2ma^2}}=\dfrac{\frac{n^2 \hbar^2 \pi^2}{2ma^2}}{\frac{\hbar^2 \pi^2}{2ma^2}}=n^2$. So we can rescale the energy by dividing by $\pi^2/2$ and the resulting value should be the quantum number squared. This makes it easy to see at a glance how close our numerical approximate answer is the analytical answer.

In [ ]:
ϵ=[E[i]/(hbar**2*np.pi**2/(2*m*a**2)) for i in range(50)] # we only need divide by pi**2/2 but hbar, m, and a are set equal to 1 so...
for i in range(50):
  print("ϵ_{}={:>8.3f}".format(i+1,ϵ[i]))

Our calculation gives almost what we expect. Each energy is essentially $n^2$ as it should be. If you change $N$ above and re-run the cells, you can see what affect using fewer (or more) points will have on runtime and accuracy. Try N=100 and N=5000. Remember that our calculation uses an N by N matrix so N=5000 will have to deal with a 25,000,000 element matrix.

We now want to plot the wavefunctions. We can plot individual wavefunctions using matplotlib (abbreviated as $plt$) and the $x$ and $psi$ lists we made above. For instance,
```
plt.plot(x,psi[2])
```
will the wavefunction corresponding to $n=3$ (remember we start counting at 0, so $n=1$ corresponds to psi[0]).

Try plotting several of the wavefunctions below. Do they match what you think they should look like?

You can beautify your graphs by adding axis labels and a title.

```
plt.xlabel('x')
plt.ylabel('$\psi$')
plt.title('Particle in an infinite box wavefunction')
```
If you just type the lines shown here, some strange symbols will appear above the graph. Use ```plt.show()``` at the end and these go away.

Now we normalize the eigenvectors. To do this we take the integral $\int^∞_{-\infty}A^*\psi ^*A\psi dx=1$ where $A$ is the normalization constant. Solving for $A$ gives $$A=\dfrac{1}{$\int^∞_{-\infty}\psi ^*\psi dx=1} $$. In numpy, we can use the trapezoidal integration function, ```trapz``` to do the integration.

In [10]:
norm=[np.sqrt(1/np.trapz(psi[i]**2,x)) for i in range(50)] # calculate normalization for each eigenfunction
psi_norm=[psi[i]*norm[i] for i in range(50)] # normalize psi

We'll set up a mechanism to plot a bunch of wavefunctions at the same time.

The sign of the wavefunctions as we approach the ends is arbitrary. We can make it consistent if we want. The $for$ loop below plots each wavefunction but it also makes it so that the left end points all approach zero from above. The right end points then alternate.

In [ ]:
plt.figure(figsize=(10,7))
fact=0.05 # scale for plotting
nl=9
for i in range(1,nl+1): # start with i=1 and go to i=
  if psi_norm[i-1][10] < 0:   # Flip the wavefunctions if it is negative at large x, so plots are more consistent.
      plt.plot(x,-psi_norm[i-1]/np.sqrt(Δx)*fact+ϵ[i-1],label="$ϵ_{}$={:>8.3f}".format(i,ϵ[i-1]))
  else:
      plt.plot(x,psi_norm[i-1]/np.sqrt(Δx)*fact+ϵ[i-1],label="$ϵ_{}$={:>8.3f}".format(i,ϵ[i-1]))
plt.title("Eigenfunctions of the Infinite Square Well")
V_x = np.zeros([N],float) # potential energy is just zeros. We don't need it for calculating but we do for plotting
for i in range(N): # but adjust potential energy at the ends where it is infinite
    if x[i] == -a/2 or x[i]==a/2:
        V_x[i]=1.35*ϵ[nl-1] # we can't plot ininity but extend the box to 135% of the highest plotted energy level
plt.plot(x,V_x)
plt.legend()
plt.savefig("Infinite_Square_Well_WaveFunctions.pdf")
plt.show()

We now also want to check that the energy levels do indeed correspond to the known levels:
$$
E_n = \frac{n^2 \pi^2 \hbar^2}{2 m a^2}
$$

In [ ]:
for i in range(nl):
    n = i+1
    print("ϵ[{}] = {:9.4f},     ϵ_{} ={:9.4f},      %deviation={:>8.4f}%".format(n,ϵ[i],n, n*n,((ϵ[i]-n*n)/n/n)))

A final test shows the accuracy of our calculation in the orthonormality of the states:

In [13]:
np.trapz(psi_norm[1]**2,x) # check that it works for one psi

1.0

The code above checks the normalization of the n=2 eigenfunctions. We can check the orthonormality of a range of levels using the following lines.

In [ ]:
for j in range(nl):
    for i in range(nl):
        print("ψ_{}, ψ_{}, {:2.2f}".format(i+1,j+1,np.trapz(psi_norm[i]*psi_norm[j],x)))

Explain the results in terms of orthonormality.




## Particle in a larger box

Let's compare that to a particle in a box that is twice as large. Again we will calculate energies relative to the lowest level in the original box.

We set up variables the same way as before. (We don't really need to redefine variables if they're not changing but it's more clear this way.)

In [15]:
hbar=1 # use atomic units
m=1 # electron mass
N = 1000

a2 = 2 # the width now is twice what it was before.
x = np.linspace(-a2/2.,a2/2.,N)

Δx = x[1]-x[0] # Should be equal to 2*np.pi/(N-1). This is delta x.
nl= 5 # the number of levels to calculate

The operators will be the same as before.


In [16]:
V = 0.*x # could be just zero but put x here to show that it is a function of x.
D2 = 1./(Δx**2)*(np.diag(np.ones(N-1),-1) -2* np.diag(np.ones(N),0) + np.diag(np.ones(N-1),1)) # 2nd derivative operator
H = -(hbar*hbar)/(2.0*m)*D2 + np.diag(V)   # Hamiltonian including derivative and potential energy.
                                            # Again, the V is unnecessary because

E2,psiT = np.linalg.eigh(H) # This computes the eigenvalues and eigenvectors of the hamiltonian matrix
psi = np.transpose(psiT)    # We take the transpose of psiT to the wavefunction vectors can accessed as psi[n]
                            # The eigenfunctions are returned in psiT as rows. We can access a row with the slightly
                            # more complicated notation psiT[::,n] but renaming the transpose of the matrix makes it simpler.

In [ ]:
ϵ2=[E2[i]*2/np.pi**2 for i in range(nl)]
print("energy level       energy in original box       energy in 2x box       ratio")
for i in range(nl):
  print("ϵ_{}=            {:>8.3f}                     {:>8.3f}               {:>8.3f}".format(i+1,ϵ[i],ϵ2[i],ϵ2[i]/ϵ[i]))

Do these results make sense? Explain.


Without calculating, would happen if the box were 1/2 as wide as the original box? Calculate it.


## Particle in a finite box

Now we turn our attention to a problem without an easy analytical solution.
The particle in a finite box is a similar problem but the box doesn't have infinite sides.

In [18]:
hbar=1 # Planck's constant in atomic units.
m=1 # mass of electron
N = 2000 # number of points to calculate

a = 1.0 # width of finite box
b=8 # total width of system
x = np.linspace(-b/2.,b/2.,N) # x-coordinate

V_out=400 #potential energy in atomic units

nl=9 #number of levels to calculate

lftlimit=np.searchsorted(x,[-a/2])[0]  # find the index of the left edge of the box to be used in integration later.
                                      #searchsorted used here looks through x and finds the value closest to -a/2.
                                      # We want to store step size, this is the reliable way:
Δx = x[1]-x[0] # Should be equal to 2*np.pi/(N-1)

Now we set up the potential. Here we want the potential to be zero between a/2 and -a/2 and $V_0$ outside that range.

In [19]:
V=[]
for i in x:
  if i<-a/2 or i>a/2:   # if outside the box
    V.append(V_out)     # potential is V_out
  else:
    V.append(0)         # inside it's 0
#print(V)

Now setup the operators and calculate. Everything is the same here

In [20]:
D2 = 1./(Δx**2)*(np.diag(np.ones(N-1),-1) -2* np.diag(np.ones(N),0) + np.diag(np.ones(N-1),1)) # 2nd derivative
H = -(hbar*hbar)/(2.0*m)*D2 + np.diag(V) # Hamiltonian

Ef,psiT = np.linalg.eigh(H) # This computes the eigen values and eigenvectors
psi = np.transpose(psiT)  # We take the transpose of psiT to the wavefunction vectors can accessed as psi[n]

Ef=Ef/(hbar**2 * np.pi**2/2/m/a**2) # scale energy relative to standard particle in an infinite box. hbar, m, and a are set to 1
                                  # so all energies are in units of hbar**2/(m*a**2)
V=[V/(hbar**2 * np.pi**2/2/m/a**2) for V in V] # same for potential energy.

In [ ]:
norm=[np.sqrt(1/np.trapz(psi[i]**2,x)) for i in range(nl)]  # calcluate normalization coefficient (sqrt of 1/integral of psi squared)
                                                            # Should be same but no cost to calculate for each level.
                                                            # trapz is numpy's (trapezoidal) integration function. input function and the variable
for i in range(0,nl): # start with i=1 and go to i=
  if psi[i][lftlimit] < 0:   # Flip the wavefunctions if it is negative at large x, so plots are more consistent.
      psi_norm[i]=-psi[i]*norm[i]
  else:
      psi_norm[i]=psi[i]*norm[i-1]

plt.plot(x,psi_norm[2]+Ef[2])# plot third wavefunction to make sure it looks right.
plt.ylim(-1,12) # change or comment out with a # if you want to look at other eigenfunctions
plt.xlim(-2,2)
plt.plot(x,V)
plt.show()


Check to make sure the wavefunctions are normalized.

As before, let's plot several wavefunctions on the same graph offset by their energies. It should be emphasized that all particles are travelling only on the $x$-axis. Offsetting along the $y$-axis only gets them out of each other's way and shows where the energy levels are.

In [ ]:
#plot several wavefunctions on same graph with potential. Each one placed at its energy level.
plt.figure(figsize=(5,8))
for i in range(0,nl):
  plt.plot(x,psi_norm[i]+Ef[i],label="$E_{}$={:>8.3f}".format(i+1,Ef[i]))
plt.title("Eigenfunctions of the Finite Square Well")

plt.plot(x,V)
plt.xlim(-a*.8,a*.8)
#plt.legend()
#plt.savefig("Finite_Square_Well_WaveFunctions.pdf")
plt.show()


Do the same thing for the square of the wavefunction.

In [ ]:
#plot square of wavefunction
plt.figure(figsize=(5,8))
for i in range(nl):
    plt.plot(x,psi_norm[i]**2+Ef[i],label="$E_{}$={:>8.3f}".format(i+1,Ef[i])) # make a plot for each level and label it
plt.title("Finite Well Wavefunctions Squared")
plt.plot(x,V)
plt.legend()
plt.xlim(-.8*a,.8*a)
#plt.savefig("Finite_Square_Well_WaveFunctions_Squared.pdf")
plt.show()

In [ ]:
ϵ2=[Ef[i]*2/np.pi**2 for i in range(nl)]
print("energy level       energy in infinite box       energy in finite box       ratio")
for i in range(nl):
  print("ϵ_{}=            {:>8.3f}                     {:>8.3f}               {:>8.3f}".format(i+1,ϵ[i],Ef[i],Ef[i]/ϵ[i]))

We can see that $\psi$ does not go to zero at the end of the box the way that it did in the infinite box. This is especially noticeable for the higher levels. We can integrate $\psi$ from a/2 to b (i.e. the region on the right outside the box) and multiply by 2 to get the total area on both sides. The $trapz$ integration function for numerical trapezoidal integration takes for arguments $y$ and $x$. In our case we'll take $psi$ for the $i$th level from the right boundary to the end (psi[i][rgtlimit:]) and square it, normalize it, and integrate.

In [ ]:
outob=[np.trapz((psi_norm[i][:lftlimit])**2,x[:lftlimit])*2 for i in range(0,nl)] #integrate from the left edge of the box to -'infinity' and multipy by 2 to account for both sides
for i in range(0,nl):
      print("n={}, E[{}] = {:9.4f},         Probability to be outside box={:7.3f},              Percent outside box={:5.1f}%".format(i+1,i+1,E[i],outob[i],outob[i]*100))

## Particle in an asymmetric box

What if the potential is infinite on one side only.

In [26]:
hbar=1 # Planck's constant in atomic units.
m=1 # mass of electron
N = 2000 # number of points to calculate

a = 1.0 # width of finite box
b=10 # total width of system
x = np.linspace(0,b,N) # x-coordinate
rgtlimit=np.searchsorted(x,[a/2])[0] # find the index of the right edge of the box to be used in integration later.
                                    #searchsorted used here looks through x and finds the value closest to a/2.
Δx = x[1]-x[0]                      #step size

V_out=400 #potential energy in atomic units

V=[]                                #potential -
for i in x:
  if i>a:
    V.append(V_out)                 # potential on the right
  else:
    V.append(0)                     # potential in the box
                                    # potential on the left is infinite which
#print(V)

D2 = 1./(Δx**2)*(np.diag(np.ones(N-1),-1) -2* np.diag(np.ones(N),0) + np.diag(np.ones(N-1),1))
H = -(hbar*hbar)/(2.0*m)*D2 + np.diag(V)

Ea,psiT = np.linalg.eigh(H) # This computes the eigen values and eigenvectors
psi = np.transpose(psiT)  # We take the transpose of psiT to the wavefunction vectors can accessed as psi[n]

norm=[np.sqrt(1/np.trapz(psi[i]**2,x)) for i in range(50)] #calcluate normalization coefficient. Should be same but no cost to calculate for each level.
Ea=Ea/np.pi**2*2 # scale energy relative to standard particle in an infinite box
V=[V/np.pi**2*2 for V in V] # same for potential energy.

nl=9
psi_norm=[psi[i]*norm[i] for i in range(nl)] # normalize

In [ ]:
#nl=9 #number of levels to calculate
fact=1
#plot square of wavefunction
V_x =V # potential energy is just zeros. We don't need it for calculating but we do for plotting
for i in range(N): # but adjust potential energy at the ends where it is infinite
    if x[i] == 0:
        V_x[i]=1.35*Ea[nl] # we can't plot ininity but extend the box to 135% of the highest plotted energy level
plt.figure(figsize=(6,8))

#nl=9 #number of levels to calculate
for i in range(nl):
    plt.plot(x,psi_norm[i]**2*fact**2+Ea[i],label="$E_{}$={:>8.3f}".format(i+1,Ea[i]))
plt.title("Finite Well Wavefunctions Squared")
plt.plot(x,V_x,color='black')
plt.xlim(-a/2,b/4)
plt.legend(loc="lower right")
#plt.savefig("Finite_Square_Well_WaveFunctions_Squared.pdf")
plt.show()

In [ ]:
ϵ2=[Ea[i]*2/np.pi**2 for i in range(nl)]
print("energy level       energy in infinite box       energy in finite box       ratio")
for i in range(nl):
  print("ϵ_{}=            {:>8.3f}                     {:>8.3f}               {:>8.3f}".format(i+1,ϵ[i],Ea[i],Ea[i]/ϵ[i]))


In [ ]:
outob=[np.trapz((psi_norm[i][rgtlimit:])**2,x[rgtlimit:]) for i in range(0,nl)] #integrate from the right edge of the box to 'infinity'
for i in range(0,nl):
      print("n={}, E[{}] = {:9.4f},         Probability to be outside box={:7.3f},              Percent outside box={:5.1f}%".format(i+1,i+1,E[i],outob[i],outob[i]*100))

##Double square well
What if we have two finite square wells separated by a barrier?

In [30]:
hbar=1 # Planck's constant in atomic units.
m=1 # mass of electron
N = 2000 # number of points to calculate

a = 1.0 # width of finite box
bar=3 # with of barrier
b=50 # total width of system

V0=200 #potential energy in atomic units
Vb=200 #height of barrier
nl=20 #number of levels to calculate
x = np.linspace(-b/2.,b/2.,N) # x-coordinate
rgtlimit=np.searchsorted(x,[a/2])[0] # find the index of the right edge of the box to be used in integration later.
#searchsorted used here looks through x and finds the value closest to a/2.
# We want to store step size, this is the reliable way:
h = x[1]-x[0] # Should be equal to 2*np.pi/(N-1)

In [31]:
V=[]
for i in x:
  if i>-bar/2 and i<bar/2:
    V.append(Vb)
  elif (i>(-bar/2-a) and i<-bar/2) or (i<(bar/2+a) and i>bar/2):
    V.append(0)
  else:
    V.append(V0)
#print(V)

In [32]:
D2 = 1./(h*h)*(np.diag(np.ones(N-1),-1) -2* np.diag(np.ones(N),0) + np.diag(np.ones(N-1),1))
H = -(hbar*hbar)/(2.0*m)*D2 + np.diag(V)

Ed,psiT = np.linalg.eigh(H) # This computes the eigenvalues and eigenvectors
psi = np.transpose(psiT)  # We take the transpose of psiT to the wavefunction vectors can accessed as psi[n]
norm=[np.sqrt(1/np.trapz(psi[i]**2,x)) for i in range(nl)] #calcluate normalization coefficient. Should be same but no cost to calculate for each level.
psi_norm=[psi[i]*norm[i] for i in range(nl)] # normalize

Ed=Ed/np.pi**2*2 # scale energy relative to standard particle in an infinite box
V=[V/np.pi**2*2 for V in V] # same for potential energy.

In [ ]:
#plot several wavefunctions on same graph with potential. Each one placed at its energy level.
fact=1 #scale factor for nice plotting. 1 does nothing but if the plots need to be scaled up or down, change it.
plt.figure(figsize=(12,8))
for i in range(nl):
    if psi[i][50] < 0:   # Flip the wavefunctions if it is negative at small x, so plots are more consistent.
        plt.plot(x,-psi_norm[i]*fact+Ed[i],label="$E_{}$={:>8.3f}".format(i+1,Ed[i]))
    else:
        plt.plot(x,psi_norm[i]*fact+Ed[i],label="$E_{}$={:>8.3f}".format(i+1,Ed[i]))
    plt.title("Solutions to the Finite Double Square Well")

plt.plot(x,V)
plt.xlim(-(a+bar/1.5),(a+bar/1.5))
#plt.legend()
#plt.savefig("Finite_Square_Well_WaveFunctions.pdf")
plt.show()

In [ ]:
#plot square of wavefunction
plt.figure(figsize=(12,8))
for i in range(nl):
    plt.plot(x,psi_norm[i]**2*fact**2+Ed[i],label="$E_{}$={:>8.3f}".format(i+1,Ed[i]))
plt.title("Finite Double Well Wavefunctions Squared")
plt.plot(x,V)
plt.xlim(-a-bar/1.5,a+bar/1.5)
plt.legend(loc="center")
#plt.savefig("Finite_Square_Well_WaveFunctions_Squared.pdf")
plt.show()

Let's look at the probability density for each wavefunction. Move the slider to see each level's energy and wavefunction separately.

In [ ]:
# @title Double Well Probability { run: "auto" }
l = 2 #@param {type:"slider", min:0, max:15, step:1}
#plot square of wavefunction
plt.figure(figsize=(12,6))
plt.plot(x,psi_norm[l]**2*fact**2+Ed[l],label="$E_{}$={:>8.3f}".format(l+1,Ed[l]))
plt.title("Finite Double-Well Wavefunctions Squared")
plt.plot(x,V)
plt.xlim(-a-bar/1.5,a+bar/1.5)
plt.legend()
#plt.savefig("Finite_Square_Well_WaveFunctions_Squared.pdf")
plt.show()

What happens when we decrease the width of the barrier?

#Appendices


#####Matrix Multiplication
If you have not taken linear algebra it will be useful to see the mechanics of multplication of a small matrix with a vector. We will use a 3x3 matrix, $O$ (for $operator$) and a 3x1 vector, $f$ (for $function$) as examples.
$$O=\begin{bmatrix}1&0&3\\0&5&4\\2&6&0\end{bmatrix}\,\,\,\,\,\,\,\,\,\,\,f=\begin{bmatrix} 1\\2\\3\end{bmatrix}$$
Multiplication of the vector by the matrix leads to a 3x1 vector. The first element comes from the sum of the top leftmost element multiplied by the first element of the vector, the top-center element multiplied of the matrix by the second element of the vector and the top rightmost element of the matrix multiplied by the last element of the vector. In other words the first element of the product is the sum of the elemnts on the first row of the matrix multiplied by the first (only) column of the vector.

$$Of=\begin{bmatrix}1&0&3\\0&5&4\\2&6&0\end{bmatrix}\begin{bmatrix} 1\\2\\3\end{bmatrix}=\begin{bmatrix}1\cdot1+0\cdot2+3\cdot3\\0\cdot1+5\cdot2+4\cdot3\\2\cdot1+6\cdot2+0\cdot3\end{bmatrix}=\begin{bmatrix}10\\22\\14\end{bmatrix}$$
In this case our vector is not an eigenvector of the matrix because it doesn't emerge from the multiplication as a constant times the original vector.
Multiplying two matrices is similar but we won't worry about that now.

We can do this matrix multiplication in python too.

In [36]:
import numpy as np # load package needed for dealing with matrices and numerical calculations.

In [ ]:
O=np.array([[1,0,3],[0,5,4],[2,6,0]])
#print(O) # if you want to see O, remove the first # and run the cell
f=np.array([[1],[2],[3]])
O.dot(f) # np.dot(O,f) works too

Try multiplying $$\begin{bmatrix} 1&-1&0\\-1&1&-1\\ 0&-1&1\end{bmatrix}$$ by $$\begin{bmatrix}2\\3\\1\end{bmatrix}$$

## np.diag and np.ones
To make a derivative matrix with ones and minus ones on some diagonals, it is convenient to use numpy's ```diag``` function. ```np.diag([1,2,3])``` makes a matrix with 1, 2, and 3 along the diagonal and zeros elsewhere. Likewise, we can insert elements along a *subdiagonal* or *superdiagonal* (the diagonal below or above the true diagonal of a square matrix) using ```np.diag([1,2,3],1]```



In [ ]:
np.diag([1,2,3])

In [ ]:
np.diag([1,2,3],-1)

We have seen that a matrix with -1 on the diagonal just above the diagonal will make a derivative operator.  We can do this using ```np.diag([-1,-1,-1,-1,-1,-1])``` which works fine if we know just how big our matrix is going to be. But if we want to adjust things, we'll have to remember to adjust the number of 1's in our diagonal matrix.

We can make use of ```np.ones``` to generate a list of ones that is adjustable. So ```np.diag(-np.ones(5))``` will give a 5x5 matrix with -1 on the diagonal and ```np.diag(np.ones(4),1)``` will give a 5x5 matrix with +1 on the superdiagonal. These can be added to give the deriviative matrix we want. Substituting a variable such as $N$ and $N-1$ instead of 5 and 4 will make it so that just changing the value of $N$ will adjust the size of the derivative matrix so we can handle whatever size f(x) vector we want.

[back to derivative calculation](#scrollTo=MsgIOf7cEFkv&line=1&uniqifier=1)

##forward and backward derivatives
When we say forward derivative, we're talking about the technique of taking the derivative of a discretized function by taking finding the slope using the difference between $f(x+Δx)$ and $f(x)$ wheras a backward deriviate would use $f(x)$ and $f(x-Δx)$. There's no reason to prefer one over the other. The main effect of the choice will be whether to ignore the first or the last element of the result since there's no $f(x+Δx)$ at the last element and no $f(x-Δx)$ at the first element. We get to choose.

When calculating the second derivative, we could do each derivative forward which would result in a derivative that would look like $$
\frac{d^2}{dx^2}f(x) = \lim_{\Delta x \rightarrow 0} \frac{f'(x+\Delta x)-f'(x)}{\Delta x} =  \lim_{\Delta x \rightarrow 0} \frac{f(x+\Delta x+\Delta x) - f(x+\Delta x) - (f(x+\Delta x) - f(x))}{\Delta x^2} \\
\frac{d^2}{dx^2}f(x) = \lim_{\Delta x \rightarrow 0} \frac{f(x+2\Delta x) - 2f(x+\Delta x) + f(x))}{\Delta x^2} \approx \frac{f(x+2h) - 2f(x+h) + f(x))}{h^2}
$$
This second derivative is really leaning forward enough that we should consider it a derivative around $x+Δx$ rather than around $x$. Therefore it makes sense to take the second derivative in the opposite direction as we took the first. Since we chose to do a forward first derivative, we choose a backward second derivative. This means that both the first element and last elements of our resulting 2nd derivative vector are meaningless.

[Back to the second order differential section](#scrollTo=eEvfX6Yb7WRX&line=6&uniqifier=1)